In [ ]:
pip install -e git+https://github.com/phonghnguyen/PyHyperScattering.git#egg=PyHyperScattering

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import PyHyperScattering
import os
import shutil
import psutil
import pickle
import imageio
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib as mpl
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from matplotlib import font_manager
from matplotlib.colors import LogNorm
from matplotlib.colorbar import Colorbar as colorbar
from IPython.display import clear_output

In [ ]:
# variables to store directories, enables easy switching between systems
font_dirs = ['/nsls2/users/pnguyen2/Fonts/']

base_data_dirs = ['/nsls2/data/sst/legacy/RSoXS/suitcased_data/users/2022-3/309585-UCSB/auto/Blends/2022-09-27/',
             '/nsls2/data/sst/legacy/RSoXS/suitcased_data/users/2022-3/309585-UCSB/auto/Dopant/2022-09-27/',
             '/nsls2/data/sst/legacy/RSoXS/suitcased_data/users/2022-3/309585-UCSB/auto/Dopant/2022-09-28/']

In [ ]:
font_files = font_manager.findSystemFonts(fontpaths = font_dirs)

for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

# font_names = [f.name for f in font_manager.fontManager.ttflist]
# print(font_names)
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Avenir'
plt.rcParams['font.size'] = 18
plt.rcParams['axes.linewidth'] = 2

cmap = mpl.cm.viridis
cmap.set_bad((68/255, 1/255, 84/255), 1)

In [ ]:
def get_immediate_subdirectories(a_dir):
    
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

def fmt(x, pos):
    a, b = '{:.0e}'.format(x).split('e')
    b = int(b)
    
    return r'${} \times 10^{{{}}}$'.format(a, b)

def find_edge(scan):
    if np.min(scan.energy) == 270:
        current_edge = 'C 1s'
        current_edge_folder = 'C1s'
    elif np.min(scan.energy) == 385:
        current_edge = 'N 1s'
        current_edge_folder = 'N1s'
    elif np.min(scan.energy) == 670:
        current_edge = 'F 1s'
        current_edge_folder = 'F1s'
    else:
        current_edge = 'unknown'
        current_edge_folder = 'unknown'
        
    return [current_edge, current_edge_folder]

def match_pols(scans):
    '''
    Iterates through list of scans in scans (made by appending scans to an initally empty list). 
    First scan in list is used as a reference and the corresponding scan of orthogonal polarization is found.
    As matches are found, items from the list are removed and returned, along with the scans that were matched.
    The intention is that the list is fed back to this function to find the next pair to process until the list is empty.
    '''
    # Case for finding 0 pol at list beginning, find matching 90 pol
    if scans[0].unstack('system').polarization == 0:
        current_scan_0pol = scans[0]
        del scans[0]
        [current_edge_0pol, current_edge_folder_0pol] = find_edge(current_scan_0pol)
        print(f'Scan {current_scan_0pol.sampleid} matches {current_scan_0pol.sample_name} {current_edge_0pol} ({current_scan_0pol.rsoxs_config}) with 0°  polarity')

        if len(scans) != 0:
            for i, scan in enumerate(scans):
                if scan.rsoxs_config == current_scan_0pol.rsoxs_config:
                    [current_edge_90pol, current_edge_folder_90pol] = find_edge(scan)
                    if current_edge_90pol == current_edge_0pol:
                        if scan.sample_name == current_scan_0pol.sample_name:
                            if scan.unstack('system').polarization == 90:
                                current_scan_90pol = scan
                                print(f'Scan {scan.sampleid} matches {scan.sample_name} {current_edge_90pol} ({current_scan_90pol.rsoxs_config}) with 90° polarity')
                                del scans[i]
                                return [current_scan_0pol, current_scan_90pol, scans]
            current_scan_90pol = []
        else:
            print('Reached end, unable to pair last scan')
            
    # Case for finding 90 pol at list beginning, find matching 0 pol
    if scans[0].unstack('system').polarization == 90:
        current_scan_90pol = scans[0]
        del scans[0]
        
        [current_edge_90pol, current_edge_folder_90pol] = find_edge(current_scan_90pol)
            
        print(f'Scan {current_scan_90pol.sampleid} matches {current_scan_90pol.sample_name} {current_edge_90pol} ({current_scan_90pol.rsoxs_config}) with 90° polarity')

        if len(scans) != 0:
            for i, scan in enumerate(scans):
                if scan.rsoxs_config == current_scan_90pol.rsoxs_config:
                    [current_edge_0pol, current_edge_folder_0pol] = find_edge(scan)
                    if current_edge_0pol == current_edge_90pol:   
                        if scan.sample_name == current_scan_90pol.sample_name:
                            if scan.unstack('system').polarization == 0:
                                current_scan_0pol = scan
                                [current_edge, current_edge_folder] = find_edge(current_scan_0pol)
                                print(f'Scan {scan.sampleid} matches {scan.sample_name} {current_edge_0pol} ({current_scan_0pol.rsoxs_config}) with 0°  polarity')
                                del scans[i]
                                return [current_scan_0pol, current_scan_90pol, scans]
            current_scan_0pol = []
        else:
            print('Reached end, unable to pair last scan')
            
    return [current_scan_0pol, current_scan_90pol, scans]

def return_scan(scans):
    '''
    Iterates through list of scans in scans (made by appending scans to an initally empty list). 
    Each call results in items from the list are removed and returned, along with the top scan.
    The intention is that the list is fed back to this function to find the next pair to process until the list is empty.
    '''
    current_scan = scans[0]
    del scans[0]
    [current_edge, current_edge_folder] = find_edge(current_scan)
    print(f'Scan {current_scan.sampleid} matches {current_scan.sample_name} {current_edge} ({current_scan.rsoxs_config})')
            
    return [current_scan, scans]

def assign_mask(scan):
    if scan.rsoxs_config == 'waxs':
        mask_method = 'nika'
        mask = 'SST1_Sept2022_WAXS_mask.hdf'
    elif scan.rsoxs_config == 'saxs':
        mask_method = 'none'
        mask = []
    else:
        mask_method = 'none'
        mask = []
    
    return [mask_method, mask]

def make_output_dir(base_save_dir, scan, tail):
# tail is a string that describes the data file being exported, used as folder prefix
    [current_edge, current_edge_folder] = find_edge(scan)
    
    # create directories for output files
    save_dir = os.path.join(base_save_dir, f'{tail}_{sample_name_root(scan)}')
    os.makedirs(save_dir, exist_ok = True)
    
    return save_dir

def clear_dir(dir):
    for file_name in os.listdir(dir):
        file_path = os.path.join(dir, file_name)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
            
def sample_name_root(scan):
    [current_edge, current_edge_folder] = find_edge(scan)
    return f'{current_edge_folder}_{scan.sample_name}'

def scan_loader_SST1(base_data_dirs, exclude = [], ll = [], ul = []):
    '''
    Args:
        base_data_dirs (list of str): path to the directories to load
        exlude (list of str): list of scan IDs to exclude from loading into scans
        ll (str): scan ID below which to exclude from loading into scans
        ul (str) scan ID above which to exclude from loading into scans
    '''
    base_data_dir_tracker = 0
    scan_tracker = 0
    scan_skip_corr = 0
    scans = []
    for base_data_dir in base_data_dirs:
        base_data_dir_tracker += 1
        scan_tracker = 0
        scan_skip_corr = 0
        all_scans = pd.Series(get_immediate_subdirectories(base_data_dir))
        for current_scan in np.sort(all_scans):
            scan_tracker += 1
            if ((len(ll) == 0) and (len(ul) == 0)):
                if current_scan in exclude:
                    print(f'Skipping scan {current_scan} - ID in list to exclude')
                    scan_skip_corr += 1
                else:
                    clear_output(wait = True)
                    print(f'RAM usage: {psutil.virtual_memory().available * 100 / psutil.virtual_memory().total :.2f} %')
                    print(f'CPU usage: {psutil.cpu_percent() :.2f} %')
                    print(f'Currently pulling data from \'{base_data_dir}\' ({base_data_dir_tracker}/{len(base_data_dirs)})')
                    print(f'Currently loading scan {current_scan} ({scan_tracker - scan_skip_corr}/{len(all_scans) - scan_skip_corr})')
                    scan = file_loader.loadFileSeries(base_data_dir + current_scan, ['energy', 'polarization'])
                    scans.append(scan)
            elif ((len(ll) > 0) and (len(ul) == 0)):
                if current_scan >= ll:
                    if current_scan in exclude:
                        print(f'Skipping scan {current_scan} - ID in list to exclude')
                        scan_skip_corr += 1
                    else:
                        clear_output(wait = True)
                        print(f'RAM usage: {psutil.virtual_memory().available * 100 / psutil.virtual_memory().total :.2f} %')
                        print(f'CPU usage: {psutil.cpu_percent() :.2f} %')
                        print(f'Currently pulling data from \'{base_data_dir}\' ({base_data_dir_tracker}/{len(base_data_dirs)})')
                        print(f'Currently loading scan {current_scan} ({scan_tracker - scan_skip_corr}/{len(all_scans) - scan_skip_corr})')
                        scan = file_loader.loadFileSeries(base_data_dir + current_scan, ['energy', 'polarization'])
                        scans.append(scan)
            elif ((len(ll) == 0) and (len(ul) > 0)):
                if current_scan <= ul:
                    if current_scan in exclude:
                        print(f'Skipping scan {current_scan} - ID in list to exclude')
                        scan_skip_corr += 1
                    else:
                        clear_output(wait = True)
                        print(f'RAM usage: {psutil.virtual_memory().available * 100 / psutil.virtual_memory().total :.2f} %')
                        print(f'CPU usage: {psutil.cpu_percent() :.2f} %')
                        print(f'Currently pulling data from \'{base_data_dir}\' ({base_data_dir_tracker}/{len(base_data_dirs)})')
                        print(f'Currently loading scan {current_scan} ({scan_tracker - scan_skip_corr}/{len(all_scans) - scan_skip_corr})')
                        scan = file_loader.loadFileSeries(base_data_dir + current_scan, ['energy', 'polarization'])
                        scans.append(scan)
            elif ((len(ll) > 0) and (len(ul) > 0)):
                if ((current_scan >= ll) and (current_scan <= ul)):
                    if current_scan in exclude:
                        print(f'Skipping scan {current_scan} - ID in list to exclude')
                        scan_skip_corr += 1
                    else:
                        clear_output(wait = True)
                        print(f'RAM usage: {psutil.virtual_memory().available * 100 / psutil.virtual_memory().total :.2f} %')
                        print(f'CPU usage: {psutil.cpu_percent() :.2f} %')
                        print(f'Currently pulling data from \'{base_data_dir}\' ({base_data_dir_tracker}/{len(base_data_dirs)})')
                        print(f'Currently loading scan {current_scan} ({scan_tracker - scan_skip_corr}/{len(all_scans) - scan_skip_corr})')
                        scan = file_loader.loadFileSeries(base_data_dir + current_scan, ['energy', 'polarization'])
                        scans.append(scan)

    return scans

def setup_plot():
    plt.rcParams.update({"font.size": 18})
    plt.rcParams.update({"axes.linewidth": 2})

    fig, axes = plt.subplots(1, 1, figsize = (3.5, 3.5))

    # Edit the major and minor ticks of the x and y axes
    axes.xaxis.set_tick_params(which = 'major', size = 5, width = 2, direction = 'in', top = 'on')
    axes.xaxis.set_tick_params(which = 'minor', size = 2.5, width = 2, direction = 'in', top = 'on')
    axes.yaxis.set_tick_params(which = 'major', size = 5, width = 2, direction = 'in', right = 'on')
    axes.yaxis.set_tick_params(which = 'minor', size = 2.5, width = 2, direction = 'in', right = 'on')
    
    return [fig, axes]
    
def save_fig(fig, scan, current_energy, save_dir, tail, res):
    if current_energy > 0:
        save_path = os.path.join(save_dir, f'{tail}_{sample_name_root(scan)}_{current_energy}eV.png')
        fig.savefig(save_path, bbox_inches = 'tight', pad_inches = 0.2, dpi = res, transparent = False)
    else:
        save_path = os.path.join(save_dir, f'{tail}_{sample_name_root(scan)}.png')
        fig.savefig(save_path, bbox_inches = 'tight', pad_inches = 0.2, dpi = res, transparent = False)
        
def make_gif(scan, input_dir, output_dir, tail):
    image_folder = os.fsencode(input_dir)

    file_names = []

    for file in os.listdir(image_folder):
        file_name = os.fsdecode(file)
        file_names.append(file_name)

    file_names.sort() # this iteration technique has no built in order, so sort the frames

    # Make it pause at the end so that the viewers can ponder
    for _ in range(10):
        file_names.append(file_names[-1])

    images = list(map(lambda file_name: imageio.imread(os.path.join(input_dir, file_name)), file_names))

    imageio.mimsave(os.path.join(output_dir, f'{tail}_{sample_name_root(scan)}.gif'), images, duration = 0.5) # modify duration as needed

In [ ]:
# sacrificial empty plot because the first plot does not properly update params
[fig, axes] = setup_plot()

In [ ]:
# Determined from Nika
sample_to_det_dist = 43.11
BC_x = 395.87
BC_y = 552.74

In [ ]:
current_energy = 284
current_scan = '44065'
base_path = '/nsls2/data/sst/legacy/RSoXS/suitcased_data/users/2022-3/309585-UCSB/auto/Blends/2022-09-27/'

In [ ]:
# Determined from Nika
sample_to_det_dist = 43.11
BC_x = 395.87
BC_y = 552.74

In [ ]:
file_loader = PyHyperScattering.load.SST1RSoXSLoader(corr_mode='none')

scan = file_loader.loadFileSeries(base_path + current_scan,['energy','polarization'])

if scan.rsoxs_config == 'waxs':
    maskmethod = 'nika'
    mask = 'SST1_Sept2022_WAXS_mask.hdf'
elif scan.rsoxs_config =='saxs':
    maskmethod = 'none'
else:
    maskmethod = 'none'

integ = PyHyperScattering.integrate.PFEnergySeriesIntegrator(geomethod = "nika",
                                                             NIdistance = sample_to_det_dist, 
                                                             NIbcx = BC_x, NIbcy = BC_y,
                                                             NIpixsizex = 0.06, NIpixsizey = 0.06,
                                                             correctSolidAngle = True, 
                                                             integration_method = 'csr_ocl',
                                                             npts = 200)

In [ ]:
scans = []

scans.append(scan)

print(f'{len(scans)} scans imported')

while len(scans) > 0:
    
    # reset energy
    current_energy = 0
        
    # Find samples with matching experimental configurations, elemental edges, names, and corresponding polarizations
    # also takes care of removing already processed scans from the list of scans remaining
    current_scan = []
    
    while ((len(current_scan) == 0) and (len(scans) > 0)):
        [current_scan, scans] = return_scan(scans)

    # if tail not in os.listdir(reduced_save_dir):
    if True:
    
        # Assign the mask depending on experimental config
        [mask_method, mask] = assign_mask(current_scan)

        # import mask into PyHyperScattering integrator object
        integ.loadNikaMask(filetoload = mask, rotate_image = False)

        # rotate mask to match experimental orientation
        integ.mask = np.flipud(np.rot90(integ.mask))

        # # compare experimental and mask orientation
        # f, axarr = plt.subplots(1,2)
        # axarr[0].imshow(scan_unstacked.sel(energy=np.max(scan.energy)),
        #                 # norm = LogNorm(1,np.nanpercentile(scan_unstacked.sel(energy=current_energy),99)
        #                 vmin = 0,
        #                 vmax = np.nanpercentile(scan_unstacked.sel(energy=np.max(scan.energy)), 90),
        #                 cmap = 'viridis',
        #                 aspect='1')
        # axarr[1].imshow(integ.mask, norm = LogNorm(1,np.nanpercentile(scan_unstacked.sel(energy=np.max(scan.energy)),90)))
    
        # Find the elemental edge of the current scan
        [current_edge, current_edge_folder] = find_edge(current_scan)

        print(f'RAM usage: {psutil.virtual_memory().available * 100 / psutil.virtual_memory().total :.2f} %')
        print(f'CPU usage: {psutil.cpu_percent() :.2f} %')
        
        print(f'Integrating scan {current_scan.sampleid}:')
              
        integrated_data = integ.integrateImageStack(current_scan)
        integrated_data_unstacked = integrated_data.unstack('system')
        
        print(f'RAM usage: {psutil.virtual_memory().available * 100 / psutil.virtual_memory().total :.2f} %')
        print(f'CPU usage: {psutil.cpu_percent() :.2f} %')
        
        # Set the energy and q range of interest depending on the elemental edge of the current scan
        if current_edge == 'C 1s':
            start_en = 280
            end_en = 295
            start_q = 0.01
            end_q = 0.1
        else:
            start_en = np.nanmin(integrated_data.energy)
            end_en = np.nanmax(integrated_data.energy)
            start_q = 0.01
            end_q = np.nanmax(integrated_data.q)
            
        clear_output(wait = True)
        

        
        
        
        
        
        
        
        
        
        
        
           
        ## Make plot of the AR
        para = integrated_data_unstacked.rsoxs.slice_chi(90, chi_width = 22.5).sel(q = slice(start_q, end_q,), polarization = 0)
        perp = integrated_data_unstacked.rsoxs.slice_chi(180, chi_width = 22.5).sel(q = slice(start_q, end_q), polarization = 0)

        AR = (para - perp)/(para + para)

        scale = np.nanmax([abs(np.nanpercentile(AR.sel(energy = slice(280, 295)), 1)), abs(np.nanpercentile(AR.sel(energy = slice(start_en, end_en)), 99))])
        # scale = np.nanmax([abs(np.nanmin(AR.sel(energy = slice(280, 295)))), abs(np.nanmax(AR.sel(energy = slice(start_en, end_en))))])
        
        [fig, axes] = setup_plot()

        im_cbar = AR.sel(energy = slice(start_en, end_en)).plot(
            # norm = LogNorm(1,current_vmax_cake[-1]),
            x = 'q', y = 'energy',
            ax = axes,
            vmin = -scale,
            vmax = scale,
            cmap = 'RdBu',
            add_colorbar = False)

        cbar_ax = fig.add_axes([0.975, 0.145, 0.03, 0.715])
        cbar = fig.colorbar(im_cbar, cax = cbar_ax)

        # plt.setp(cbar.ax.get_yticklabels(), ha = "right")
        # cbar.ax.tick_params(pad = 37.5)
        cbar.ax.set_ylabel('Anisotropy')

        axes.set_title(f'{current_scan.sample_name}, {current_edge} edge', fontsize = 20, y = 1, x = 0.5)
        axes.set_xlabel('$\it{q}$ (Å$^{-1}$)')
        axes.set_ylabel('Energy (eV)')
        axes.set_xlim([start_q, end_q])
        # axes.set_xticks([0.01, 0.03, 0.05, 0.07, 0.09], ['', '0.03', '0.05', '0.07', '0.09'])
        axes.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
        axes.set_ylim([start_en, end_en])

        axes.xaxis.set_tick_params(which = 'major', size = 5, width = 2, direction = 'in', top = 'on')
        axes.xaxis.set_tick_params(which = 'minor', size = 2.5, width = 2, direction = 'in', top = 'on')
        axes.yaxis.set_tick_params(which = 'major', size = 5, width = 2, direction = 'in', right = 'on')
        axes.yaxis.set_tick_params(which = 'minor', size = 2.5, width = 2, direction = 'in', right = 'on')

        plt.show()


        [fig, axes] = setup_plot()

        im_cbar = para.plot(
            # norm = LogNorm(1,current_vmax_cake[-1]),
            x = 'q', y = 'energy',
            ax = axes,
            vmin = np.nanpercentile(para, 97.5),
            vmax = scale,
            cmap = cmap,
            add_colorbar = False)
        
        cbar_ax = fig.add_axes([0.975, 0.145, 0.03, 0.715])
        fig.colorbar(im_cbar, cax = cbar_ax)

        axes.set_title(f'{current_scan.sample_name}, {current_edge} edge', fontsize = 20, y = 1, x = 0.5)
        axes.text(0.975, 0.91, f'Parallel', color = 'white', fontsize = 15, transform = axes.transAxes, horizontalalignment = 'right')

        axes.set_xlabel('$\it{q}$ (Å$^{-1}$)')
        axes.set_ylabel('Energy (eV)')
        axes.set_xlim([start_q, end_q])
        # axes.set_xticks([0.01, 0.03, 0.05, 0.07, 0.09], ['', '0.03', '0.05', '0.07', '0.09'])
        axes.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
        axes.set_ylim([start_en, end_en])

        axes.xaxis.set_tick_params(which = 'major', size = 5, width = 2, direction = 'in', top = 'on')
        axes.xaxis.set_tick_params(which = 'minor', size = 2.5, width = 2, direction = 'in', top = 'on')
        axes.yaxis.set_tick_params(which = 'major', size = 5, width = 2, direction = 'in', right = 'on')
        axes.yaxis.set_tick_params(which = 'minor', size = 2.5, width = 2, direction = 'in', right = 'on')

        plt.show()
        

        [fig, axes] = setup_plot()

        im_cbar = perp.plot(
            # norm = LogNorm(1,current_vmax_cake[-1]),
            x = 'q', y = 'energy',
            ax = axes,
            vmin = np.nanpercentile(perp, 97.5),
            vmax = scale,
            cmap = cmap,
            add_colorbar = False)

        cbar_ax = fig.add_axes([0.975, 0.145, 0.03, 0.715])
        fig.colorbar(im_cbar, cax = cbar_ax)

        axes.set_title(f'{current_scan.sample_name}, {current_edge} edge', fontsize = 20, y = 1, x = 0.5)
        axes.text(0.975, 0.91, f'Perpendicular', color = 'white', fontsize = 15, transform = axes.transAxes, horizontalalignment = 'right')

        axes.set_xlabel('$\it{q}$ (Å$^{-1}$)')
        axes.set_ylabel('Energy (eV)')
        axes.set_xlim([start_q, end_q])
        # axes.set_xticks([0.01, 0.03, 0.05, 0.07, 0.09], ['', '0.03', '0.05', '0.07', '0.09'])
        axes.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
        axes.set_ylim([start_en, end_en])

        axes.xaxis.set_tick_params(which = 'major', size = 5, width = 2, direction = 'in', top = 'on')
        axes.xaxis.set_tick_params(which = 'minor', size = 2.5, width = 2, direction = 'in', top = 'on')
        axes.yaxis.set_tick_params(which = 'major', size = 5, width = 2, direction = 'in', right = 'on')
        axes.yaxis.set_tick_params(which = 'minor', size = 2.5, width = 2, direction = 'in', right = 'on')

        plt.show()


        para_perp = para + perp

        [fig, axes] = setup_plot()

        im_cbar = para_perp.plot(
            # norm = LogNorm(1,current_vmax_cake[-1]),
            x = 'q', y = 'energy',
            ax = axes,
            vmin = np.nanpercentile(para_perp, 97.5),
            vmax = scale,
            cmap = cmap,
            add_colorbar = False)
                
        cbar_ax = fig.add_axes([0.975, 0.145, 0.03, 0.715])
        fig.colorbar(im_cbar, cax = cbar_ax)

        axes.set_title(f'{current_scan.sample_name}, {current_edge} edge', fontsize = 20, y = 1, x = 0.5)
        axes.text(0.975, 0.91, f'Circular', color = 'white', fontsize = 15, transform = axes.transAxes, horizontalalignment = 'right')

        axes.set_xlabel('$\it{q}$ (Å$^{-1}$)')
        axes.set_ylabel('Energy (eV)')
        axes.set_xlim([start_q, end_q])
        # axes.set_xticks([0.01, 0.03, 0.05, 0.07, 0.09], ['', '0.03', '0.05', '0.07', '0.09'])
        axes.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
        axes.set_ylim([start_en, end_en])

        axes.xaxis.set_tick_params(which = 'major', size = 5, width = 2, direction = 'in', top = 'on')
        axes.xaxis.set_tick_params(which = 'minor', size = 2.5, width = 2, direction = 'in', top = 'on')
        axes.yaxis.set_tick_params(which = 'major', size = 5, width = 2, direction = 'in', right = 'on')
        axes.yaxis.set_tick_params(which = 'minor', size = 2.5, width = 2, direction = 'in', right = 'on')

        plt.show()
        

        [fig, axes] = setup_plot()

        FY_NEXAFS = integrated_data_unstacked.mean('chi').sel(q = slice(0.055, 0.075), energy = slice(start_en, end_en)).mean('q')
        
        FY_NEXAFS.plot(
            ax = axes,
            color = 'k')

        axes.set_title(f'{current_scan.sample_name}, {current_edge} edge', fontsize = 20, y = 1, x = 0.5)
        axes.set_ylabel('Intensity (a.u.)')
        axes.set_xlabel('Energy (eV)')
        axes.set_xlim([start_en, end_en])
        axes.set_ylim([0.9*np.nanmin(FY_NEXAFS), 
                       1.1*np.nanmax(FY_NEXAFS)])

        plt.show()

        
        
        
                
        
    

        
        
        
        
        

print(f'{len(scans)} scans leftover')